In [1]:
import sys
sys.path.insert(0, '..')
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/model")
import clip
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from sklearn.metrics import top_k_accuracy_score
from sklearn.model_selection import train_test_split
import pickle
from tqdm import tqdm
import gc
import os
import pickle
from model.aliproduct_model import ALIPRODUCT_CLIP
from model.aliproduct_blip_model import ALIPRODUCT_BLIP
from model.dataset import ALIPRODUCT_DATASET,prepare_data
from model.CONFIG import CONFIG
import faiss
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode


In [ ]:


'''Edit this to specify checkpoints of models you want to test 
defined as key value pair of {"checpoint name/identifier:"full path to checkpoint"}'''

checkpoints = {"base":"https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_retrieval_coco.pth",
"epoch_1":"/home/ubuntu/Desktop/base_3_itm/base_3rd_e_1.pth",
"epoch_2":"/home/ubuntu/Desktop/base_3_itm/base_3rd_e_2.pth",
"epoch_3":"/home/ubuntu/Desktop/base_3_itm/base_3rd_e_3.pth",
"epoch_4":"/home/ubuntu/Desktop/base_3_itm/base_3rd_e_4.pth",
"epoch_5":"/home/ubuntu/Desktop/base_3_itm/base_3rd_e_5.pth"}

#Edit this to specify the augmented/original captions you want to test the model checkpoints on 
caption_to_test =  ["caption"]


#path to validation data after preporcessing into csv file format 
dataframe_path = "/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/data/val_data_prompt_clean.csv"

In [6]:
image_size = 384
preprocess = transforms.Compose([
        transforms.Resize((image_size,image_size),interpolation=InterpolationMode.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
        ]) 


In [7]:
def test_feature(caption_col,clip_model):
    test_loader,df = prepare_data("/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/data/val_data_prompt_clean.csv",
    CONFIG.test_image_data_dir,CONFIG.test_image_data_folder
    ,CONFIG.test_image_col,caption_col,160,preprocess,CONFIG.global_random_state,test=True,tokenize=False)
    trainer = Trainer(gpus=1)
    pred = trainer.predict(clip_model,test_loader)
    full_pred = tuple(map(torch.concat, zip(*pred)))
    image_embed,text_embed = full_pred
    # print(image_embed.size())
    # print(text_embed.size())
    faiss_index = faiss.IndexFlatIP(256)
    print(image_embed.numpy().astype(np.float32).shape)
    faiss_index.add(image_embed.numpy().astype(np.float32))
    top5_k_e,top5_k_y_pred = faiss_index.search(text_embed.numpy().astype(np.float32),5)
    top10_k_e,top10_k_y_pred = faiss_index.search(text_embed.numpy().astype(np.float32),10)
    top5_preds = np.array([1 if y_true in y_pred else 0 for y_true,y_pred in zip(df.index.values,top5_k_y_pred)])
    print(caption_col,"num correct pred",sum(top5_preds))
    top10_preds = np.array([1 if y_true in y_pred else 0 for y_true,y_pred in zip(df.index.values,top10_k_y_pred)])
    print(caption_col,"num correct pred",sum(top10_preds))
    top5_acc = top5_preds[top5_preds ==1].shape[0] / top5_preds.shape[0]
    top10_acc = top10_preds[top10_preds ==1].shape[0] / top10_preds.shape[0]
    return top5_acc,top10_acc

In [8]:

top5_acc_preds = []
top10_acc_preds = []
captions_tested = []
for cap in caption_to_test:
    for checkpoint in checkpoints.keys():
        clip_model =ALIPRODUCT_BLIP(checkpoints[checkpoint],image_size)
        top5_acc,top10_acc = test_feature(caption_to_test,clip_model)
        top5_acc_preds.append(top5_acc)
        top10_acc_preds.append(top10_acc)


load checkpoint from https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_retrieval_coco.pth


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting DataLoader 0: 100%|██████████| 313/313 [18:32<00:00,  3.55s/it]
(50000, 256)
caption num correct pred 6806
caption num correct pred 9587
load checkpoint from /home/ubuntu/Desktop/base_3_itm/base_3rd_e_1.pth


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting DataLoader 0: 100%|██████████| 313/313 [18:32<00:00,  3.55s/it]
(50000, 256)
caption num correct pred 22711
caption num correct pred 29021
load checkpoint from /home/ubuntu/Desktop/base_3_itm/base_3rd_e_2.pth


...

Predicting DataLoader 0: 100%|██████████| 313/313 [18:59<00:00,  3.64s/it]
(50000, 256)
caption num correct pred 24159
caption num correct pred 30524
load checkpoint from /home/ubuntu/Desktop/base_3_itm/base_3rd_e_4.pth


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting DataLoader 0: 100%|██████████| 313/313 [18:56<00:00,  3.63s/it]
(50000, 256)
caption num correct pred 24812
caption num correct pred 31197
load checkpoint from /home/ubuntu/Desktop/base_3_itm/base_3rd_e_5.pth


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting DataLoader 0: 100%|██████████| 313/313 [18:52<00:00,  3.62s/it]
(50000, 256)
caption num correct pred 25424
caption num correct pred 31777


In [9]:
pred_df = {"model_checkpoint":checkpoints.keys(),"top_5":top5_acc_preds,"top_10":top10_acc_preds}

In [10]:
top5_acc_preds

[0.13612, 0.45422, 0.46924, 0.48318, 0.49624, 0.50848]

In [11]:
pred_df = pd.DataFrame(pred_df)

In [12]:
pred_df

,model_checkpoint,top_5,top_10
0,base,0.13612,0.19174
1,epoch_1,0.45422,0.58042
2,epoch_2,0.46924,0.59978
3,epoch_3,0.48318,0.61048
4,epoch_4,0.49624,0.62394
5,epoch_5,0.50848,0.63554
